# Fórmula Mágica - Joel Greenblat

Joel Greenblatt escreveu um livro (“The Little Book that Beats the Market”), onde explica como os investidores podem aplicar sistematicamente uma fórmula que busca bons negócios quando eles estão disponíveis a preços de pechincha. A essa estratégia deu o nome de Fórmula Mágica.

Essa fórmula usa dois indicadores:
* ROE – Return on Equity, retorno sobre Patrimônio Líquido;
* P/L – índice Preço/Lucro.

Baseado nesses indicadores queremos formar um ranking de papéis que possuam o maior ROE e o menor P/L.

Essa é a forma bem simplificada, o próprio Greenblat faz algumas ressalvas e sugere o uso dos indicadores ROIC e EV/EBIT para melhorar a fórmula. Mas aqui, para efeito de estudos de desenvolvimento em Python vou me ater a apenas essas duas regras: encontrar os 20 ativos que estejam com o maior ROE e o menor P/L e exibir essa lista.
O formato dessa carteira é anual. Ou seja, o investidor compra as ações indicadas hoje e só vai trocar por novas ações no próximo ano.

#### 1. Buscar os dados no site Fundamentus

In [7]:
import yfinance
import requests
import pandas as pd
import numpy as np

url = "https://www.fundamentus.com.br/resultado.php"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"}
r = requests.get(url, headers=headers)

lista = pd.read_html(r.text, decimal=",", thousands=".")
dados = lista[0]

#### 2. Tratar os dados

Nesta etapa vamos fazer um tratamento nos dados:
 
    * remover as colunas que não vamos utilizar e deixar apenas: Papel, ROE, P/L;
    * remover as linhas que tenham P/L muito alto ou negativo (acima de 100 ou abaixo de zero);
    * remover as linhas com ROE muito alto ou negativo (acima de 100 ou abaixo de zero);

In [8]:
#o site fundamentus também traz os outros indicadores sugeridos pelo Greenblat 
# ROIC – Quanto maior melhor EV/EBIT – Quanto menor melhor

#deletando todas as colunas que não serão utilizadas
dados.drop(columns=['Cotação', 
                    'P/VP', 
                    'PSR', 
                    'Div.Yield',
                    'P/Ativo',
                    'P/Cap.Giro',
                    'P/EBIT',
                    'P/Ativ Circ.Liq',
                    'EV/EBIT',
                    'EV/EBITDA',
                    'Mrg Ebit',
                    'Mrg. Líq.',
                    'Liq. Corr.',
                    'ROIC',
                    'Patrim. Líq',
                    'Dív.Brut/ Patrim.',
                    'Cresc. Rec.5a'], inplace=True)

#tratamento dos dados para remover as distorções
#PL negativo ou muito alto
indexNames = dados[dados['P/L'] <= 0 ].index
dados.drop(indexNames , inplace=True)
indexNames = dados[dados['P/L'] >= 100 ].index
dados.drop(indexNames , inplace=True)

#garantindo que os dados analisados estão em formato numérico
#transformando as porcentagens em valores numéricos
#não fui em quem criou essa função, mas achei muito boa e resolvi adicionar ao código
def columns_which_contains(df, value):
    """
    Serch for DataFrame column's values which contains a specific value
    :value: any characters, phrases, symbols
    :returns: list of DataFrame column's names
    """
    return [df[column].name for column in df if df[column].astype(str).str.contains(value).any()]

percentual_columns = columns_which_contains(dados, '\%')

for column_name in percentual_columns:
    dados[column_name] = dados[column_name].str.replace('.', '', regex=True)
    dados[column_name] = dados[column_name].str.replace(',', '.', regex=True)
    dados[column_name] = dados[column_name].str.replace('[%,]', '', regex=True).astype(float)
    dados[column_name] = dados[column_name]/100

#ROE negativo ou muito alto
#antes da tranformação dos dados do ROE o trecho abaixo dava erro
indexNames = dados[dados['ROE'] <= 0 ].index
dados.drop(indexNames , inplace=True)
indexNames = dados[dados['ROE'] >= 1 ].index
dados.drop(indexNames , inplace=True)

#vou adicionar também um filtro de liquidez
#deixando apenas as ações com liquidez maior que 50000 nos últimos 2 meses
indexNames = dados[dados['Liq.2meses'] < 50000].index
dados.drop(indexNames , inplace=True)


#### 3. Ranking

Agora, vamos construir um ranking das ações que se enquadram nos critérios.

A empresa com o maior ROE recebe a nota 1, a segunda recebe nota 2 e assim por diante. A mesma coisa é feita para o P/L, o menor P/L recebe nota 1, o segundo menor nota 2....

A partir dessa notas criaremos uma nova coluna com as notas SOMADAS dos dois indicadores. Aquelas com MENORES notas representam as “melhores” empresas pela combinação dos dois indicadores.

In [9]:
#ordenação dos dados de forma descendente do ROE
dados.sort_values('ROE', ascending=False, inplace=True)

#indexando os dados seguindo a ordenação descendente
#agora o maior ROE está no índice 0 e o menor ROE está no último índice
dados.index = range(0,len(dados))

dados["notaROE"] = range(0,len(dados)) 

#ordenação dos dados de forma ascendente do P/L
dados.sort_values('P/L', ascending=True, inplace=True)

dados["notaPL"] = range(0,len(dados)) 


#fazendo a soma das notas

dados["somaNotas"] = 0

dfd = dados.copy()

for i in dfd.index:
    dfd.loc[i, 'somaNotas'] = dfd.loc [i, 'notaPL']  + dfd.loc[i, 'notaROE']

dfd.sort_values('somaNotas', ascending=True, inplace=True)

print("20 ativos - Fórmula Greenblat\n")
print(dfd.iloc[0:21])

20 ativos - Fórmula Greenblat

    Papel   P/L     ROE    Liq.2meses  notaROE  notaPL  somaNotas
6   SYNE3  0.73  0.7727  5.956910e+06        6       1          7
5   SUZB3  3.09  0.8578  3.721510e+08        5      15         20
4   TASA4  3.20  0.8601  2.326890e+07        4      19         23
3   TASA3  3.23  0.8601  1.068270e+06        3      21         24
26  USIM3  1.29  0.4068  8.780440e+06       26       2         28
27  USIM5  1.39  0.4068  2.456890e+08       27       3         30
24  ETER3  3.02  0.4182  1.959120e+07       24      12         36
9   VALE3  3.46  0.6377  2.683240e+09        9      28         37
30  GGBR3  2.31  0.3772  1.439140e+06       30       7         37
12  JBSS3  3.40  0.5461  4.411210e+08       12      26         38
34  GOAU3  2.18  0.3571  7.233020e+05       34       6         40
31  GGBR4  2.92  0.3772  4.008800e+08       31      11         42
35  GOAU4  2.33  0.3571  1.067380e+08       35       8         43
19  DEXP4  3.36  0.4221  5.521880e+04       1

#### 4. Resultado

Esse é o método sistemático e sem influência de nenuma análise um pouco mais refinada. Nem mesmo eu me aprofundei nos indicadores usados na fórmula, se está tudo correto ou não (se alguém quiser contibuir e me corrigir sobre qualquer coisa, fique a vontade).

Será que é um bom método usar apenas o ROE e o P/L para nos indicar a compra de ações? Talvez um P/L muito baixo não queira dizer apenas que a empresa esteja muito barata, pode ser que o mercado não tenha confiança na companhia. Assim como um ROE alto pode não significar que os negócios estão rentáveis.

Mas Greenblat criou essa Fórmula Mágica e será que ela não é rentável? Que tal um backtest de como seria o desempenho dessa carteira nos útimos anos?
Esse será o próximo desafio....

![Logo do Markdown](resultado.png)